In [ ]:
pip install pyspark

     |████████████████████████████████| 204.2MB 79kB/s 
     |████████████████████████████████| 204kB 49.3MB/s 
  Created wheel for pyspark: filename=pyspark-3.0.1-py2.py3-none-any.whl size=204612243 sha256=80e86644d89e4853db7eb5e24e970bfa6a5fc8309bd6f35ae86faa8f760e9ae5
  Stored in directory: /root/.cache/pip/wheels/5e/bd/07/031766ca628adec8435bb40f0bd83bb676ce65ff4007f8e73f
Successfully built pyspark


In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("heart_disease").getOrCreate()
spark

In [ ]:
df = spark.read.csv("/content/heart.csv", inferSchema=True, header=True)
df.show(2)
df.count()

+---+---+---+--------+----+---+-------+-------+-----+-------+-----+---+----+------+
|age|sex| cp|trestbps|chol|fbs|restecg|thalach|exang|oldpeak|slope| ca|thal|target|
+---+---+---+--------+----+---+-------+-------+-----+-------+-----+---+----+------+
| 63|  1|  3|     145| 233|  1|      0|    150|    0|    2.3|    0|  0|   1|     1|
| 37|  1|  2|     130| 250|  0|      1|    187|    0|    3.5|    0|  0|   2|     1|
+---+---+---+--------+----+---+-------+-------+-----+-------+-----+---+----+------+
only showing top 2 rows



303

In [ ]:
df.groupBy('target').count().show()

+------+-----+
|target|count|
+------+-----+
|     1|  165|
|     0|  138|
+------+-----+



In [ ]:
from pyspark.ml.feature import VectorAssembler, StringIndexer, MinMaxScaler
from pyspark.sql.types import * 

from pyspark.ml.classification import *
from pyspark.ml.evaluation import *
from pyspark.sql.functions import *
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

In [ ]:
def MLClassifierDFPrep(df,input_columns,dependent_var,treat_outliers=True,treat_neg_values=True):
    
    renamed = df.withColumn("label_str", df[dependent_var].cast(StringType()))
    indexer = StringIndexer(inputCol="label_str", outputCol="label")  
    indexed = indexer.fit(renamed).transform(renamed)
    print(indexed.groupBy(dependent_var,"label").count().show(100))

    
    numeric_inputs = []
    string_inputs = []
    for column in input_columns:
        if str(indexed.schema[column].dataType) == 'StringType':
            indexer = StringIndexer(inputCol=column, outputCol=column+"_num") 
            indexed = indexer.fit(indexed).transform(indexed)
            new_col_name = column+"_num"
            string_inputs.append(new_col_name)
        else:
            numeric_inputs.append(column)
            
    if treat_outliers == True:
        print("We are correcting for non normality now!")
        
        d = {}
       
        for col in numeric_inputs: 
            d[col] = indexed.approxQuantile(col,[0.01,0.99],0.25) 
      
        for col in numeric_inputs:
            skew = indexed.agg(skewness(indexed[col])).collect()
            skew = skew[0][0]
           
            if skew > 1:
                indexed = indexed.withColumn(col, \
                log(when(df[col] < d[col][0],d[col][0])\
                .when(indexed[col] > d[col][1], d[col][1])\
                .otherwise(indexed[col] ) +1).alias(col))
                print(col+" has been treated for positive (right) skewness. (skew =)",skew,")")
            elif skew < -1:
                indexed = indexed.withColumn(col, \
                exp(when(df[col] < d[col][0],d[col][0])\
                .when(indexed[col] > d[col][1], d[col][1])\
                .otherwise(indexed[col] )).alias(col))
                print(col+" has been treated for negative (left) skewness. (skew =",skew,")")

            
  
    minimums = df.select([min(c).alias(c) for c in df.columns if c in numeric_inputs]) 
    min_array = minimums.select(array(numeric_inputs).alias("mins"))
    df_minimum = min_array.select(array_min(min_array.mins)).collect() 
    df_minimum = df_minimum[0][0] 

    features_list = numeric_inputs + string_inputs
    
    assembler = VectorAssembler(inputCols=features_list,outputCol='features')
    output = assembler.transform(indexed).select('features','label')
 
    if df_minimum < 0:
        print(" ")
        print("WARNING: The Naive Bayes Classifier will not be able to process your dataframe as it contains negative values")
        print(" ")
    
    if treat_neg_values == True:
        print("You have opted to correct that by rescaling all your features to a range of 0 to 1")
        print(" ")
        print("We are rescaling you dataframe....")
        scaler = MinMaxScaler(inputCol="features", outputCol="scaledFeatures")

        scalerModel = scaler.fit(output)

        scaled_data = scalerModel.transform(output)
        final_data = scaled_data.select('label','scaledFeatures')
        final_data = final_data.withColumnRenamed('scaledFeatures','features')
        print("Done!")
    else:
        print("You have opted not to correct that therefore you will not be able to use to Naive Bayes classifier")
        print("We will return the dataframe unscaled.")
        final_data = output
    
    return final_data

In [ ]:
def ClassTrainEval(classifier,features,classes,folds,train,test):
    
    def FindMtype(classifier):
        # Intstantiate Model
        M = classifier
        # Learn what it is
        Mtype = type(M).__name__
        
        return Mtype
    
    Mtype = FindMtype(classifier)
    

    def IntanceFitModel(Mtype,classifier,classes,features,folds,train):
        
        if Mtype == "OneVsRest":
          
            lr = LogisticRegression()
            OVRclassifier = OneVsRest(classifier=lr)
            paramGrid = ParamGridBuilder() \
                .addGrid(lr.regParam, [0.1, 0.01]) \
                .build()
            
            crossval = CrossValidator(estimator=OVRclassifier,
                                      estimatorParamMaps=paramGrid,
                                      evaluator=MulticlassClassificationEvaluator(),
                                      numFolds=folds) # 3 is best practice
            
            fitModel = crossval.fit(train)
            return fitModel
        if Mtype == "MultilayerPerceptronClassifier":
            
            features_count = len(features[0][0])
            layers = [features_count, features_count+1, features_count, classes]
            MPC_classifier = MultilayerPerceptronClassifier(maxIter=100, layers=layers, blockSize=128, seed=1234)
            fitModel = MPC_classifier.fit(train)
            return fitModel
        if Mtype in("LinearSVC","GBTClassifier") and classes != 2: 
            print(Mtype," could not be used because PySpark currently only accepts binary classification data for this algorithm")
            return
        if Mtype in("LogisticRegression","NaiveBayes","RandomForestClassifier","GBTClassifier","LinearSVC","DecisionTreeClassifier"):
  
            
            if Mtype in("LogisticRegression"):
                paramGrid = (ParamGridBuilder() \

                             .addGrid(classifier.maxIter, [10, 15,20])
                             .build())
                
        
            if Mtype in("NaiveBayes"):
                paramGrid = (ParamGridBuilder() \
                             .addGrid(classifier.smoothing, [0.0, 0.2, 0.4, 0.6]) \
                             .build())
         
            if Mtype in("RandomForestClassifier"):
                paramGrid = (ParamGridBuilder() \
                               .addGrid(classifier.maxDepth, [2, 5, 10])

                             .build())
                
          
            if Mtype in("GBTClassifier"):
                paramGrid = (ParamGridBuilder() \
                             .addGrid(classifier.maxIter, [10, 15,50,100])
                             .build())
                
          
            if Mtype in("LinearSVC"):
                paramGrid = (ParamGridBuilder() \
                             .addGrid(classifier.maxIter, [10, 15]) \
                             .addGrid(classifier.regParam, [0.1, 0.01]) \
                             .build())
          
            if Mtype in("DecisionTreeClassifier"):
                paramGrid = (ParamGridBuilder() \
                             .addGrid(classifier.maxBins, [10, 20, 40, 80, 100]) \
                             .build())

            crossval = CrossValidator(estimator=classifier,
                                      estimatorParamMaps=paramGrid,
                                      evaluator=MulticlassClassificationEvaluator(),
                                      numFolds=folds) 
            fitModel = crossval.fit(train)
            return fitModel
    
    fitModel = IntanceFitModel(Mtype,classifier,classes,features,folds,train)
    

    if fitModel is not None:
        
        if Mtype in("OneVsRest"):
   
            BestModel = fitModel.bestModel
            global OVR_BestModel
            OVR_BestModel = BestModel
            print(" ")
            print('\033[1m' + Mtype + '\033[0m')

            models = BestModel.models
            for model in models:
                print('\033[1m' + 'Intercept: '+ '\033[0m',model.intercept)
                print('\033[1m' + 'Top 20 Coefficients:'+ '\033[0m')
                coeff_array = model.coefficients.toArray()
                coeff_scores = []
                for x in coeff_array:
                    coeff_scores.append(float(x))
    
                result = spark.createDataFrame(zip(input_columns,coeff_scores), schema=['feature','coeff'])
                print(result.orderBy(result["coeff"].desc()).show(truncate=False))
        if Mtype == "MultilayerPerceptronClassifier":
            print("")
            print('\033[1m' + Mtype + '\033[0m')
            print('\033[1m' + "Model Weights: "+ '\033[0m',fitModel.weights.size)
            print("")
            global MLPC_Model
            MLPC_BestModel = fitModel

        if Mtype in("DecisionTreeClassifier", "GBTClassifier","RandomForestClassifier"):
          
            BestModel = fitModel.bestModel
            print(" ")
            print('\033[1m' + Mtype," Top 20 Feature Importances"+ '\033[0m')
            print("(Scores add up to 1)")
            print("Lowest score is the least important")
            print(" ")
            featureImportances = BestModel.featureImportances.toArray()

            imp_scores = []
            for x in featureImportances:
                imp_scores.append(float(x))
            result = spark.createDataFrame(zip(input_columns,imp_scores), schema=['feature','score'])
            print(result.orderBy(result["score"].desc()).show(truncate=False))
            if Mtype in("DecisionTreeClassifier"):
                global DT_featureimportances
                DT_featureimportances = BestModel.featureImportances.toArray()
                global DT_BestModel
                DT_BestModel = BestModel
            if Mtype in("GBTClassifier"):
                global GBT_featureimportances
                GBT_featureimportances = BestModel.featureImportances.toArray()
                global GBT_BestModel
                GBT_BestModel = BestModel
            if Mtype in("RandomForestClassifier"):
                global RF_featureimportances
                RF_featureimportances = BestModel.featureImportances.toArray()
                global RF_BestModel
                RF_BestModel = BestModel

        if Mtype in("LogisticRegression"):
            BestModel = fitModel.bestModel
            print(" ")
            print('\033[1m' + Mtype + '\033[0m')
            print("Intercept: " + str(BestModel.interceptVector))
            print('\033[1m' + " Top 20 Coefficients"+ '\033[0m')
            print("You should compares these relative to eachother")
            coeff_array = BestModel.coefficientMatrix.toArray()
            coeff_scores = []
            for x in coeff_array[0]:
                coeff_scores.append(float(x))
            result = spark.createDataFrame(zip(input_columns,coeff_scores), schema=['feature','coeff'])
            print(result.orderBy(result["coeff"].desc()).show(truncate=False))
            global LR_coefficients
            LR_coefficients = BestModel.coefficientMatrix.toArray()
            global LR_BestModel
            LR_BestModel = BestModel
        if Mtype in("LinearSVC"):
            BestModel = fitModel.bestModel
            print(" ")
            print('\033[1m' + Mtype + '\033[0m')
            print("Intercept: " + str(BestModel.intercept))
            print('\033[1m' + "Top 20 Coefficients"+ '\033[0m')
            print("You should compares these relative to eachother")

            coeff_array = BestModel.coefficients.toArray()
            coeff_scores = []
            for x in coeff_array:
                coeff_scores.append(float(x))
           
            result = spark.createDataFrame(zip(input_columns,coeff_scores), schema=['feature','coeff'])
            print(result.orderBy(result["coeff"].desc()).show(truncate=False))
     
            global LSVC_coefficients
            LSVC_coefficients = BestModel.coefficients.toArray()
            global LSVC_BestModel
            LSVC_BestModel = BestModel
        
   
   
    columns = ['Classifier', 'Result']
    
    if Mtype in("LinearSVC","GBTClassifier") and classes != 2:
        Mtype = [Mtype] 
        score = ["N/A"]
        result = spark.createDataFrame(zip(Mtype,score), schema=columns)
    else:
        predictions = fitModel.transform(test)
        MC_evaluator = MulticlassClassificationEvaluator(metricName="accuracy") 
        accuracy = (MC_evaluator.evaluate(predictions))*100
        Mtype = [Mtype]
        score = [str(accuracy)]
        result = spark.createDataFrame(zip(Mtype,score), schema=columns)
        result = result.withColumn('Result',result.Result.substr(0, 5))
        
    return result
  

In [ ]:

input_columns = df.columns
input_columns = input_columns[:-1] 
dependent_var = 'target'

class_count = df.select(countDistinct("target")).collect()
classes = class_count[0][0]

In [ ]:

test1_data = MLClassifierDFPrep(df,input_columns,dependent_var)
test1_data.limit(5).toPandas()


classifiers = [
                LogisticRegression()
                ,OneVsRest()
               ,LinearSVC()
               ,NaiveBayes()
               ,RandomForestClassifier()
               ,GBTClassifier()
               ,DecisionTreeClassifier()
               ,MultilayerPerceptronClassifier()
              ] 

train,test = test1_data.randomSplit([0.8,0.2])
features = test1_data.select(['features']).collect()
folds = 3 

columns = ['Classifier', 'Result']
vals = [("Place Holder","N/A")]
results = spark.createDataFrame(vals, columns)

for classifier in classifiers:
    new_result = ClassTrainEval(classifier,features,classes,folds,train,test)
    results = results.union(new_result)
results = results.where("Classifier!='Place Holder'")
print("!!!!!Final Results!!!!!!!!")
results.show(100,False)

+------+-----+-----+
|target|label|count|
+------+-----+-----+
|     1|  0.0|  165|
|     0|  1.0|  138|
+------+-----+-----+

None
We are correcting for non normality now!
chol has been treated for positive (right) skewness. (skew =) 1.1377326187082237 )
fbs has been treated for positive (right) skewness. (skew =) 1.9768034646834516 )
oldpeak has been treated for positive (right) skewness. (skew =) 1.2634255245891595 )
ca has been treated for positive (right) skewness. (skew =) 1.303925955673585 )
You have opted to correct that by rescaling all your features to a range of 0 to 1
 
We are rescaling you dataframe....
Done!
 
LogisticRegression
Intercept: [-0.3407680837231448]
 Top 20 Coefficients
You should compares these relative to eachother
+--------+--------------------+
|feature |coeff               |
+--------+--------------------+
|ca      |3.420482697022333   |
|trestbps|3.3254111711768495  |
|thal    |2.761528949100433   |
|sex     |1.9847438603068972  |
|chol    |1.67435481138

In [ ]:
predictions = LR_BestModel.transform(test)

In [ ]:
predictions.filter("prediction==1").show(2)

+-----+--------------------+--------------------+--------------------+----------+
|label|            features|       rawPrediction|         probability|prediction|
+-----+--------------------+--------------------+--------------------+----------+
|  0.0|[0.47916666666666...|[-2.3713578842561...|[0.08538303876592...|       1.0|
|  0.0|[0.5,1.0,0.0,0.45...|[-3.5443308343590...|[0.02807686391202...|       1.0|
+-----+--------------------+--------------------+--------------------+----------+
only showing top 2 rows



In [ ]:
predictions.groupBy("label").count().show()
predictions.groupBy("prediction").count().show()

predictions.filter("prediction != label").count()

+-----+-----+
|label|count|
+-----+-----+
|  0.0|   34|
|  1.0|   25|
+-----+-----+

+----------+-----+
|prediction|count|
+----------+-----+
|       0.0|   32|
|       1.0|   27|
+----------+-----+



8

In [ ]:
from pyspark.ml.feature import VectorSlicer
from pyspark.ml.feature import ChiSqSelector
from pyspark.ml.linalg import Vectors

classifiers = [ LogisticRegression()
                ] 
n = 99

selector = ChiSqSelector(numTopFeatures=n, featuresCol="features",
                     outputCol="selectedFeatures", labelCol="label")
bestFeaturesDf = selector.fit(test1_data).transform(test1_data)
bestFeaturesDf = bestFeaturesDf.select("label","selectedFeatures")
bestFeaturesDf = bestFeaturesDf.withColumnRenamed("selectedFeatures","features")

features = bestFeaturesDf.select(['features']).collect()

train,test = bestFeaturesDf.randomSplit([0.8,0.2])

folds = 3

columns = ['Classifier', 'Result']
vals = [("Place Holder","N/A")]
results = spark.createDataFrame(vals, columns)

for classifier in classifiers:
    new_result = ClassTrainEval(classifier,features,classes,folds,train,test)
    results = results.union(new_result)
results = results.where("Classifier!='Place Holder'")
results.show(100,False)


 
LogisticRegression
Intercept: [-1.2241230117936226]
 Top 20 Coefficients
You should compares these relative to eachother
+--------+--------------------+
|feature |coeff               |
+--------+--------------------+
|ca      |3.342905220479936   |
|trestbps|2.8410007940402098  |
|thal    |2.3728834121424085  |
|chol    |1.6901700702041635  |
|sex     |1.427428712671764   |
|exang   |1.156599991524827   |
|oldpeak |1.120787406235372   |
|fbs     |-0.20321189952707713|
|age     |-0.9676058003747318 |
|restecg |-1.0465928960272672 |
|slope   |-2.2309936454218944 |
|cp      |-2.6801351887959175 |
|thalach |-3.0927961865980866 |
+--------+--------------------+

None
+------------------+------+
|Classifier        |Result|
+------------------+------+
|LogisticRegression|82.53 |
+------------------+------+

